# Tests avec NLTK

## Imports

In [3]:
import nltk
from bs4 import BeautifulSoup
import re

## Pré-traitement du PAGE XML

On récupère le PAGE XML, on le parse, et on essaye de reconstituer chaque enregistrement grâce à l'annotation sémantique réalisée dans eScriptorium.

Le *type:first_line* correspond à chaque première ligne d'enregistrement. La première ligne du registre, indiquant la date, possède également ce label.

Il me semble que le modèle de segmentation des colonnes actuellement entraîné labellise les colonnes pairs et impairs. La colonne centrale pourrait être récupéré en cherchant à extraire la troisième colonne pair. 

A chaque fin d'enregistrement, on ajoute un "@ " pour indiquer la fin de l'unité sémantique. De la sorte, on reconstitue artificiellement la structure de la page.

In [4]:
tag_list = []

# On ouvre le fichier XML PAGE
with open("../../corpus_test/lectaurep/doc_28_sample/annotated_first_lines/FRAN_0025_0029_L-0.xml", 'r', encoding='UTF-8') as file:
    parsed_sample = BeautifulSoup(file, "lxml-xml")


# On effectue les transformations pour segmenter artificiellement les enregistrements
textline_tags = parsed_sample.find_all("TextLine")
for i in range(len(textline_tags)):
    if textline_tags[i].has_attr('custom'):
        if textline_tags[i]['custom'] == 'structure {type:first_line;}':
                # On modifie l'index précédent quand on arrive sur une balise avec @custome == 'structure {type:first_line;}'
                unicode = textline_tags[i-1].find('Unicode')
                unicode.append("@ ")
      
    
# On récupère les tags Unicode modifiés
unicode_tags = parsed_sample.find_all("Unicode")
text_tags_list = [tag.text for tag in unicode_tags]
print(text_tags_list)
# On join() les index de la liste
text = ''.join(text_tags_list)
# On essaye de reconstituer les noms aglutinés à l'aide des majuscules
text = re.sub(r"(\w)([A-Z])", r"\1 \2", text)

# Pour visualisation :
text_breaklines = '\n'.join(text_tags_list)
text_breaklines = re.sub(r"(\w)([A-Z])", r"\1 \2", text_breaklines)

['An 1901, mois de Étévrier@ ', 'Dupuis (parPierre) à Paris, rue Turot 4, à Catherine Bigol, safen@ ', 'aOo (par Mve) s.n. à@ ', 'Muret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine', 'Lemaire, dt à Paris, même adreré (Sléparation de biens@ ', 'Duboscqfet titre de 380+. de rente 3% au nom de: Debuisson, Juatine Adèle', "Se de Pnles) et autres, à Paris, rue del'Orcade 21@ ", 'Rousfpa Pdmord Harcisse Bt aParis, rue Canaletti 15, et autrespr touchor V@ ', 'Chabr (par Marie Louise Jatoune, Ve deMarie Eusébe Maseine Bt à Billancourt ronte de', 'Sersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88@ ', 'Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère@ ', 'Oussonppar Geordes ErnertLéou dt à Paris, rue Lamarch 144, à sonpère@ ', "Lecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre", 'Pugute) décédé en sondomicilé à PParis, rue Descart

## Traitements NLP avec NLTK

NLTK ne propose pas de modèle de POS tagging ni de modèle de NER pour les textes en français.

Notons cependant que NLTK propose d'entraîner un modèle de POS tagging. Voir http://www.nltk.org/book/ch05.html#sec-n-gram-tagging.

On peut tout de même essayer de segmenter et tokenizer les phrases.

## Tokenizer

In [5]:
from nltk import word_tokenize

Pour reconstituer les enregistrements, on utilise un sentecizer personnalisé, configuré pour segmenter avec les "@".

Voir la documentation : https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.punkt.PunktLanguageVars.sent_end_chars

In [11]:
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [12]:
# On instancie l'objet PunkSentenceTokenizer
sentencizer = PunktSentenceTokenizer()
# On le configure pour segmenter sur les @
PunktLanguageVars.sent_end_chars = ("@")
# On applique le sentencizer sur le texte qu'on a récupéré depuis le XML PAGE
sentences = sentencizer.tokenize(text)
# On tokenize les phrases
tokens = [word_tokenize(sent, language="french") for sent in sentences]

In [13]:
print(sentences)

['An 1901, mois de Étévrier@', 'Dupuis (par Pierre) à Paris, rue Turot 4, à Catherine Bigol, safen@', 'a Oo (par Mve) s.n. à@', 'Muret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine Lemaire, dt à Paris, même adreré (Sléparation de biens@', "Duboscqfet titre de 380+. de rente 3% au nom de: Debuisson, Juatine Adèle Se de Pnles) et autres, à Paris, rue del'Orcade 21@", 'Rousfpa Pdmord Harcisse Bt a Paris, rue Canaletti 15, et autrespr touchor V@', 'Chabr (par Marie Louise Jatoune, Ve de Marie Eusébe Maseine Bt à Billancourt ronte de Sersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88@', 'Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère@', 'Oussonppar Geordes Ernert Léou dt à Paris, rue Lamarch 144, à sonpère@', "Lecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre Pugute) décédé en sondomicilé à P Paris, rue Descartes 21, le 16 8b

In [14]:
print(tokens)

[['An', '1901', ',', 'mois', 'de', 'Étévrier', '@'], ['Dupuis', '(', 'par', 'Pierre', ')', 'à', 'Paris', ',', 'rue', 'Turot', '4', ',', 'à', 'Catherine', 'Bigol', ',', 'safen', '@'], ['a', 'Oo', '(', 'par', 'Mve', ')', 's.n.', 'à', '@'], ['Muret', '(', 'de', 'Paul', 'Louis', 'Georges', ')', 'dt', 'à', 'Paris', ',', 'Bd', 'Rochechouart', '68', ',', 'et', 'Marie', 'Joséphine', 'Lemaire', ',', 'dt', 'à', 'Paris', ',', 'même', 'adreré', '(', 'Sléparation', 'de', 'biens', '@'], ['Duboscqfet', 'titre', 'de', '380+.', 'de', 'rente', '3', '%', 'au', 'nom', 'de', ':', 'Debuisson', ',', 'Juatine', 'Adèle', 'Se', 'de', 'Pnles', ')', 'et', 'autres', ',', 'à', 'Paris', ',', 'rue', "del'Orcade", '21', '@'], ['Rousfpa', 'Pdmord', 'Harcisse', 'Bt', 'a', 'Paris', ',', 'rue', 'Canaletti', '15', ',', 'et', 'autrespr', 'touchor', 'V', '@'], ['Chabr', '(', 'par', 'Marie', 'Louise', 'Jatoune', ',', 'Ve', 'de', 'Marie', 'Eusébe', 'Maseine', 'Bt', 'à', 'Billancourt', 'ronte', 'de', 'Sersailles.

On peut également essayer de récupérer les labels avec le modèle utilisé par NLTK.

In [15]:
# code snippet thanks to https://stackoverflow.com/a/45704548
for sent in sentences:
   for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
      if hasattr(chunk, 'label'):
         print(chunk.label(), ' '.join(c[0] for c in chunk))

GPE Dupuis
GPE Paris
PERSON Catherine Bigol
GPE Muret
PERSON Paul Louis Georges
GPE Paris
PERSON Bd Rochechouart
PERSON Marie
GPE Paris
GPE Duboscqfet
PERSON Debuisson
PERSON Juatine
GPE Pnles
GPE Paris
PERSON Rousfpa
PERSON Pdmord Harcisse Bt
GPE Paris
GPE Chabr
PERSON Marie Louise Jatoune
PERSON Ve
PERSON Marie
PERSON Maseine Bt
PERSON Billancourt
PERSON Marceline Fhalino
PERSON Billancourt
GPE Gauther
PERSON Marie Cexandinte
GPE Paris
PERSON Oussonppar
PERSON Geordes Ernert
GPE Paris
GPE Lecrosnier
PERSON Caistte
GPE Paris
PERSON Pierre Pugute
PERSON P Paris
PERSON Louis Cmile
PERSON Villneuse Heorges
GPE Oupont
PERSON Anepon
PERSON Cchille Pareau
GPE Mondicta
ORGANIZATION Joaguina Polores Conceptionde Orbeta
PERSON Alfred Slain Geordes
GPE Paris
GPE Deeuit
GPE Paris
PERSON Marie Gertade Cbout
GPE Mert
GPE Soudietat
PERSON Ste
ORGANIZATION Blocq
PERSON Gabiel Hanit Coyatrue
GPE Henriy
PERSON Etugustine Victoire Lecharpentier
PERSON Jean Baptiste
PERSON Moreau
ORGANIZATION Simond
PER

## Stanford NLP taggers

Il est possible d'importer les taggers mis à disposition par le Stanford NLP Group. Voir https://nlp.stanford.edu/software/.

### Stanford POS Tagger

La page du tagger POS de Stanford est disponible ici : https://nlp.stanford.edu/software/tagger.shtml
        
Pour l'utiliser, il est nécessaire d'en avoir une copie en local.

Pour l'utiliser à l'aide de Python et NLTK, voir la documentation https://github.com/nltk/nltk/blob/develop/nltk/tag/stanford.py#L138

In [2]:
# On importe la classe StanfordPOSTagger agissant comme une interface pour utiliser le tagger en local
from nltk.tag import StanfordPOSTagger

In [184]:
# Code snippet thanks to https://stackoverflow.com/a/44478073
import os

# On indique le chemin du .jar pour le tagger
jar = '/home/hugoscheithauer/Dev/NER/ner_python/ner_nltk/stanford_pos_tagger/stanford-postagger-full-2020-11-17/stanford-postagger-4.2.0.jar'
# On indique le chemin du modèle utilisé par le tagger
model = '/home/hugoscheithauer/Dev/NER/ner_python/ner_nltk/stanford_pos_tagger/stanford-postagger-full-2020-11-17/models/french-ud.tagger'
# On indique le chemin vers le JDK
java_path = "/usr/lib/jvm/java-14-openjdk-amd64/bin/java"
os.environ['JAVAHOME'] = java_path

# On instancie l'objet StanfordPOSTagger, en passant en paramètre le modèle, et le chemin vers le tagger
pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
# On récupère le POS dans une liste de tuples (token, POS)
res = [pos_tagger.tag(sent.split()) for sent in sentences]
print(res)

[[('An', 'DET'), ('1901,', 'NUM'), ('mois', 'NOUN'), ('de', 'ADP'), ('Étévrier@', 'PROPN')], [('Dupuis', 'PROPN'), ('(par', 'PROPN'), ('Pierre)', 'PROPN'), ('à', 'PROPN'), ('Paris,', 'PROPN'), ('rue', 'NOUN'), ('Turot', 'PROPN'), ('4,', 'NUM'), ('à', 'SYM'), ('Catherine', 'PROPN'), ('Bigol,', 'PROPN'), ('safen@', 'PROPN')], [('a', 'AUX'), ('Oo', 'X'), ('(par', 'X'), ('Mve)', 'X'), ('s.n.', 'X'), ('à@', 'X')], [('Muret', 'PROPN'), ('(de', 'PROPN'), ('Paul', 'PROPN'), ('Louis', 'PROPN'), ('Georges)', 'PROPN'), ('dt', 'X'), ('à', 'X'), ('Paris,', 'X'), ('Bd', 'X'), ('Rochechouart', 'PROPN'), ('68,', 'NUM'), ('et', 'CCONJ'), ('Marie', 'PROPN'), ('Joséphine', 'PROPN'), ('Lemaire,', 'PROPN'), ('dt', 'X'), ('à', 'X'), ('Paris,', 'X'), ('même', 'X'), ('adreré', 'X'), ('(Sléparation', 'NOUN'), ('de', 'ADP'), ('biens@', 'NOUN')], [('Duboscqfet', 'PROPN'), ('titre', 'NOUN'), ('de', 'ADP'), ('380+.', 'NUM'), ('de', 'ADP'), ('rente', 'NOUN'), ('3%', 'X'), ('au', 'X'), ('nom', 'NOUN'), ('

## NER 

La même opération est réalisable avec le tagger NER de Stanford NLP. 

In [5]:
from nltk.tag import StanfordNERTagger

Cependant, aucun modèle pré-entraîné de NER pour le français n'est disponible. Il est cependant possible d'en entraîner un.

Voir la documentation.

* https://nlp.stanford.edu/software/CRF-NER.html
* https://nlp.stanford.edu/software/crf-faq.html
* https://medium.com/sicara/train-ner-model-with-nltk-stanford-tagger-english-french-german-6d90573a9486
* https://medium.com/@subshakya591/custom-trained-nltk-stanford-ner-tagger-and-spacy-ner-tagger-comparison-d4e20e41b0bf

In [6]:
StanfordNERTagger

nltk.tag.stanford.StanfordNERTagger

Malheureusement, un problème est survenu durant l'entraînement d'un modèle de NER avec le tagger. L'entraînement se déroule normalement, mais aucun modèle n'est créé en output. 